# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [103]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os


## Lectura de ficheros de datos

In [104]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Read the data

First, look in the text:

In [105]:
# Read, then decode for py2 compat.
text = open(datos_abascal, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 22573 characters


In [106]:
# Take a look at the first 250 characters in text
print(text[:250])

Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas, si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para impedir que VOX siga cruzando las líneas que dice ust


In [107]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [108]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [109]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [110]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[44, 45, 46, 47, 48, 49, 50], [66, 67, 68]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [111]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [112]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [113]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [114]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


# Fases propuestas para la elaboración del modelo

### 1. Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [115]:
text = open(datos_abascal, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

### 2. Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [116]:
seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
    print(repr(''.join(idx2char[item.numpy()])))
    
    
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
    

'Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas,'
' si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para imped'
'ir que VOX siga cruzando las líneas que dice usted que cruzamos? Conteste también lo que no me ha con'
'testado durante toda esta legislatura: ¿por qué mintió a los españoles prometiéndoles que no pactaría'
' con estos, con esos y con aquellos?  Contésteme a eso y entonces no habrá monólogos, habrá diálogo p'
'olítico, algo que nunca ha habido en esta Cámara por su culpa.\nNada, señor Sánchez, le prestamos los '
'siete segundos que nos quedan para su próxima cumbre bilateral, que seguro que los aprovecha bien. Mu'
'chas gracias.\nSeñor Sánchez, no puede haber barrios ni calles seguras si no hay fronteras seguras. ¿U'
'sted sabe lo que ha pasado en España las últimas semanas? ¿Sabe usted que algunos de los yihadistas d'
'etenidos en Barcelona, que pretendían atentar en nuestro país

## 3. Build The Model

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

In [117]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [118]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim,
                  batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True,
             recurrent_initializer='glorot_uniform',
             stateful=True),
        Dense(vocab_size)
    ])
    
    return model

vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()
 

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (64, None, 512)           41472     
                                                                 
 lstm_8 (LSTM)               (64, None, 2048)          20979712  
                                                                 
 dense_8 (Dense)             (64, None, 81)            165969    
                                                                 
Total params: 21187153 (80.82 MB)
Trainable params: 21187153 (80.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [119]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [120]:
model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## 4. Summary y fit del modelo



In [121]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200


3/3 [==============================] - 2s 265ms/step - loss: 4.6340
Epoch 2/200
3/3 [==============================] - 1s 244ms/step - loss: 5.0840
Epoch 3/200
3/3 [==============================] - 1s 254ms/step - loss: 3.8636
Epoch 4/200
3/3 [==============================] - 1s 249ms/step - loss: 3.0622
Epoch 5/200
3/3 [==============================] - 1s 245ms/step - loss: 3.0155
Epoch 6/200
3/3 [==============================] - 1s 251ms/step - loss: 2.9920
Epoch 7/200
3/3 [==============================] - 1s 244ms/step - loss: 2.9540
Epoch 8/200
3/3 [==============================] - 1s 245ms/step - loss: 2.9441
Epoch 9/200
3/3 [==============================] - 1s 241ms/step - loss: 2.9070
Epoch 10/200
3/3 [==============================] - 1s 243ms/step - loss: 2.8717
Epoch 11/200
3/3 [==============================] - 1s 241ms/step - loss: 2.8281
Epoch 12/200
3/3 [==============================] - 1s 297ms/step - loss: 2.7846
Epoch 13/200
3/3 [==============================]

Aconsejable el uso de GPU en Google Colab, aunque yo he conseguido hacer 200 épocas de mi modelo en un tiempo razonable (menos de 1 hora).

**RECUERDA GRABAR EL MODELO ENTRENADO PARA PODER REUTILIZARLO POSTERIORMENTE**

## 5. Genera texto y evalúa su calidad

In [122]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [123]:
def generate_text(model, start_string):
    
    num_generate = 500
    input_eval = [char2idx[s] for s in start_string]
    
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [124]:
print(generate_text(model, start_string=u"Señor Abascal, es usted un facha fascista"))


Señor Abascal, es usted un facha fascistas españoles y al de muchos diputados de distintos grupos de esta Cámara dicientras palabras. Yo no le voy a pedir que me respete cuando usted no respeta ni a los jueces ni al jefenderal y usted diguila para su patrido nuestro país en ochenta años, el que ha provocado la mayor contracción del producto interior barrios para a pactar con Podemos y con los separatistas y por qué lo ha hecho justo al día siguiente de las electorel se findalmente exonómio e la fabricación de vehículos en España, la vo


## 6. Trabajo adicional

Por ejempl, poner en cadena los tres modelos para que "dialoguen" entre sí